<h1><center>Prepare Data for Recommendation System</center></h1>

I'm going to build a simple Neighborhood-based Recommendation System which have the same theory with KNN

Item-item Collaborative Filtering

Import library

In [1]:
import pandas as pd 
import numpy as np 
import os
from scipy import sparse 
from sklearn.metrics.pairwise import cosine_similarity
from datetime import time, timedelta 

Import Data

In [2]:
path = 'D:\Giáo Trình\Kỳ 5\Kho và Khai Phá Dữ Liệu\Project'
review_data = pd.read_csv(os.path.join(path, 'Data\\Processed Data\\review_data.csv'))
review_data.head()

,review_id,order_id,customer_unique_id,order_purchase_timestamp,review_score,review_comment_message,review_creation_date
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,68a5590b9926689be4e10f4ae2db21a8,2018-01-11 15:30:49,4,NaN,2018-01-18 00:00:00
1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,68a5590b9926689be4e10f4ae2db21a8,2018-01-11 15:30:49,4,NaN,2018-01-18 00:00:00
2,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,64190b91b656ab8f37eb89b93dc84584,2018-02-28 12:25:19,5,NaN,2018-03-10 00:00:00
3,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,1d47144362c14e94ccdd213e8ec277d5,2018-02-03 09:56:22,5,NaN,2018-02-17 00:00:00
4,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,c8cf6cb6b838dc7a33ed199b825e8616,2017-04-09 17:41:13,5,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00


In [3]:
order_data = pd.read_csv(os.path.join(path, 'Data\\Processed Data\\order_data.csv'))
order_data.head()

,order_id,order_item_id,customer_unique_id,product_id,seller_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,price,freight_value,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,871766c5855e863f6eccc05f988b23cb,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,delivered,2017-09-13 08:59:02,2017-09-20 23:43:48,2017-09-29 00:00:00,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,eb28e67c4c0b83846050ddfb8a35d051,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,delivered,2017-04-26 10:53:06,2017-05-12 16:04:24,2017-05-15 00:00:00,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,3818d81c6709e39d06b2738a8d3a2474,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,delivered,2018-01-14 14:33:31,2018-01-22 13:19:16,2018-02-05 00:00:00,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,af861d436cfc08b2c2ddefd0ba074622,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,delivered,2018-08-08 10:00:35,2018-08-14 13:32:39,2018-08-20 00:00:00,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,64b576fb70d441e8f1b2d7d446e483c5,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,delivered,2017-02-04 13:57:51,2017-03-01 16:42:31,2017-03-17 00:00:00,199.90,18.14,218.04


In [4]:
cols_need = ['customer_unique_id', 'product_id', 'review_score']
df = pd.merge(left = order_data, right = review_data, how = 'inner', on = 'customer_unique_id')[cols_need].groupby(
                ['customer_unique_id', 'product_id']).mean().reset_index()
df.head()

,customer_unique_id,product_id,review_score
0,0000366f3b9a7992bf8c76cfdf3221e2,372645c7439f9661fbbacfd129aa92ec,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5099f7000472b634fea8304448d20825,4.0
2,0000f46a3911fa3c0805444483337064,64b488de448a5324c4134ea39c28a34b,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,2345a354a6f2033609bbf62bf5be9ef6,4.0
4,0004aac84e0df4da2b147fca70cf8255,c72e18b3fe2739b8d24ebf3102450f37,5.0


Run code below to create sparse matrix, i just used first 50k rows because lack of memory

In [5]:
df = df[:50000]
df

,customer_unique_id,product_id,review_score
0,0000366f3b9a7992bf8c76cfdf3221e2,372645c7439f9661fbbacfd129aa92ec,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,5099f7000472b634fea8304448d20825,4.0
2,0000f46a3911fa3c0805444483337064,64b488de448a5324c4134ea39c28a34b,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,2345a354a6f2033609bbf62bf5be9ef6,4.0
4,0004aac84e0df4da2b147fca70cf8255,c72e18b3fe2739b8d24ebf3102450f37,5.0
...,...,...,...
49995,7d414b6a1cf56a6bc5035f9eb436b15d,526f833e1eb0a32ad9351145a761b9ab,5.0
49996,7d4237ca3e15d16434f782badc94e724,f27aff266ad97e75d4a24c893b2c311e,5.0
49997,7d4266b26a0546c7abb0cacbeb1a19f1,07821918ef487fd64eb20b3c8aeb897f,5.0
49998,7d4297ce106c752018d700d5b26b1a2a,c3f6113d5b61bc95468432072b27e23d,1.0


In [6]:
# denseMatrix = df.groupby(['product_id', 'customer_unique_id'])['review_score'].mean().unstack().fillna(0)

In [7]:
# matrix = sparse.lil_matrix(denseMatrix.values)

In [8]:
# matrix.maxprint = matrix.shape[0]
# with open("spare_matrix.txt","w") as file:
#     file.write(str(matrix)) 
#     file.close() 

Calculate items similarity

In [9]:
# similarity = cosine_similarity(matrix)

In [10]:
# mat = sparse.lil_matrix(similarity)

In [11]:
# mat.maxprint = mat.shape[0]
# with open("similarity.txt","w") as file:
#     file.write(str(mat)) 
#     file.close() 

Because we'll be working on sparse matrix in txt file which does not use 'product_id' and 'customer_id' but row id instead, so we need to create a dictionary to encode 'product_id' and 'customer_id' to row id to apply model

Create dateframe to contain customer id and its alternative id

In [12]:
#customer_id = df[['customer_unique_id']].drop_duplicates()
#customer_id['alternative_customer_id'] = range(0, customer_id.shape[0])

In [13]:
#customer_id.to_csv(path + '\\Web App\\Data\\customer_id.csv', index = False)
#customer_id

Create dataframe to contain product id and its alternative id

In [14]:
#indexMatrix = denseMatrix.reset_index()

In [15]:
# product_id = indexMatrix[['product_id']]
# product_id['alternative_product_id'] = range(0, product_id.shape[0])

In [16]:
# product_id[['product_id', 'alternative_product_id']].to_csv(path + '\\Web App\\Data\\product_id.csv', index = False)
# product_id

Prepare Data for Recommendation Strategy

In [17]:
colsNeed = ['product_id', 'order_purchase_timestamp_x', 'price', 'review_score']
recommendData = pd.merge(left = order_data, right = review_data, how = 'inner', on = 'customer_unique_id')[colsNeed]
recommendData

,product_id,order_purchase_timestamp_x,price,review_score
0,4244733e06e7ecb4970a6e2683c13e61,2017-09-13 08:59:02,58.90,5
1,e5f2d52b802189ee658865ca93d83a8f,2017-04-26 10:53:06,239.90,4
2,e5f2d52b802189ee658865ca93d83a8f,2017-04-26 10:53:06,239.90,5
3,d2f5484cbffe4ca766301b21ab9246dd,2017-09-18 16:12:04,12.88,4
4,d2f5484cbffe4ca766301b21ab9246dd,2017-09-18 16:12:04,12.88,5
...,...,...,...,...
171780,4aa6014eceb682077f9dc4bffebc05b0,2018-04-23 13:57:06,299.99,5
171781,32e07fd915822b0765e448c4dd74c828,2018-07-14 10:26:46,350.00,5
171782,72a30483855e2eafc67aee5dc2560482,2017-10-23 17:07:56,99.90,5
171783,9c422a519119dcad7575db5af1ba540e,2017-08-14 23:02:59,55.99,5


In [18]:
recommendData['order_purchase_timestamp_x'] = pd.to_datetime(recommendData['order_purchase_timestamp_x'])
df1 = recommendData.copy()
df2 = recommendData.copy()
df1.head()

,product_id,order_purchase_timestamp_x,price,review_score
0,4244733e06e7ecb4970a6e2683c13e61,2017-09-13 08:59:02,58.90,5
1,e5f2d52b802189ee658865ca93d83a8f,2017-04-26 10:53:06,239.90,4
2,e5f2d52b802189ee658865ca93d83a8f,2017-04-26 10:53:06,239.90,5
3,d2f5484cbffe4ca766301b21ab9246dd,2017-09-18 16:12:04,12.88,4
4,d2f5484cbffe4ca766301b21ab9246dd,2017-09-18 16:12:04,12.88,5


In [19]:
df1['month'] = df1['order_purchase_timestamp_x'].dt.month
del df1['order_purchase_timestamp_x']
del df1['price']
df1.head()

,product_id,review_score,month
0,4244733e06e7ecb4970a6e2683c13e61,5,9
1,e5f2d52b802189ee658865ca93d83a8f,4,4
2,e5f2d52b802189ee658865ca93d83a8f,5,4
3,d2f5484cbffe4ca766301b21ab9246dd,4,9
4,d2f5484cbffe4ca766301b21ab9246dd,5,9


In [20]:
df1 = df1.groupby(['month', 'product_id']).agg(count = pd.NamedAgg(column="product_id", aggfunc="count"),
                    review_score = pd.NamedAgg(column="review_score", aggfunc="mean")).sort_values(
                    ['month', 'product_id', 'count', 'review_score'], ascending = [True, True, True, False]).reset_index()

df1.head()

,month,product_id,count,review_score
0,1,001c5d71ac6ad696d22315953758fa04,1,5.0
1,1,002959d7a0b0990fe2d69988affcbc80,3,5.0
2,1,0043c62d00db47eff6a6bc4cf6bfaeda,1,1.0
3,1,005030ef108f58b46b78116f754d8d38,1,2.0
4,1,005c6b24cc96dca3e2c01e824401030e,1,5.0


In [21]:
del df2['order_purchase_timestamp_x']
del df2['review_score']
df2.head()

,product_id,price
0,4244733e06e7ecb4970a6e2683c13e61,58.90
1,e5f2d52b802189ee658865ca93d83a8f,239.90
2,e5f2d52b802189ee658865ca93d83a8f,239.90
3,d2f5484cbffe4ca766301b21ab9246dd,12.88
4,d2f5484cbffe4ca766301b21ab9246dd,12.88


In [22]:
df2 = df2.groupby('product_id')['price'].mean().reset_index()
df2.head()

,product_id,price
0,00066f42aeeb9f3007548bb9d3f33c38,101.65
1,00088930e925c41fd95ebfe695fd2655,129.90
2,0009406fd7479715e4bef61dd91f2462,229.00
3,000b8f95fcb9e0096488278317764d19,58.90
4,000d9be29b5207b54e86aa1b1ac54872,199.00


In [23]:
immeRecommendData = pd.merge(left = df1, right = df2, how = 'inner', on = 'product_id')
immeRecommendData

,month,product_id,count,review_score,price
0,1,001c5d71ac6ad696d22315953758fa04,1,5.000000,79.900000
1,1,002959d7a0b0990fe2d69988affcbc80,3,5.000000,129.900000
2,1,0043c62d00db47eff6a6bc4cf6bfaeda,1,1.000000,527.990000
3,1,005030ef108f58b46b78116f754d8d38,1,2.000000,14.545556
4,2,005030ef108f58b46b78116f754d8d38,4,4.000000,14.545556
...,...,...,...,...,...
60791,12,ff415a8a6bfbaf8244f4ee0c7c30d905,2,1.000000,133.600000
60792,12,ff5ecb2f7628793844e861076ab8e467,1,1.000000,85.500000
60793,12,ff7ac89ca5b77d0fb5f8a65262d73956,1,3.000000,97.990000
60794,12,ffbc83054b3741a8d67fc59d9cf9d42d,3,2.333333,44.900000


In [25]:
category = pd.read_csv(os.path.join(path, 'Data\\Processed Data\\product_review_score.csv'))
category = category[['product_id', 'product_category_name_english']]
finalRecommendData = pd.merge(left = immeRecommendData, right = category, on = 'product_id', how = 'inner')

In [26]:
finalRecommendData['review_score'] = finalRecommendData['review_score'].round(2)
finalRecommendData['price'] = finalRecommendData['price'].round(2)
finalRecommendData

,month,product_id,count,review_score,price,product_category_name_english
0,1,001c5d71ac6ad696d22315953758fa04,1,5.00,79.90,bed_bath_table
1,1,002959d7a0b0990fe2d69988affcbc80,3,5.00,129.90,furniture_decor
2,1,0043c62d00db47eff6a6bc4cf6bfaeda,1,1.00,527.99,sports_leisure
3,1,005030ef108f58b46b78116f754d8d38,1,2.00,14.55,perfumery
4,2,005030ef108f58b46b78116f754d8d38,4,4.00,14.55,perfumery
...,...,...,...,...,...,...
59865,12,ff415a8a6bfbaf8244f4ee0c7c30d905,2,1.00,133.60,toys
59866,12,ff5ecb2f7628793844e861076ab8e467,1,1.00,85.50,toys
59867,12,ff7ac89ca5b77d0fb5f8a65262d73956,1,3.00,97.99,perfumery
59868,12,ffbc83054b3741a8d67fc59d9cf9d42d,3,2.33,44.90,housewares


In [28]:
finalRecommendData.to_csv(path + '\Web App\Data\\recommendData.csv', index = False)